# CIFAR 100 rescon

We want to find out how to maximize the perforamnce given a total amount of memory. 

Chart Layout:

* x-axis: total memory (#slots is varied)
* y-axis: acc
* legend: different models

The following experiments might be interesting: How much do different compression techniques change the performance of the architecture?

1. Baseline without compression
2. Thinning
3. Quantization
4. Convolutional Autoencoding
5. Fully Connected Autoencoding

Each with ResNet-34 or CutR34(3)

In [ ]:
BACKBONE_BLOCK = 3
SEED = 0
LOG_DIR = '/home/marwei/code/encodedgdumb/logs/cifar100_rescon'
DATA_DIR = '/home/marwei/pytorch'
mem_sizes = [10, 100, 1000, 10000]

In [ ]:
from pathlib import Path

def write_save(name, contents):
    out_path = Path('..', 'scripts', name).resolve()
    if out_path.exists():
        print('File already exits, nothing has been overwritten')
    else:
        with open(out_path, 'w') as f:
            f.write(contents)

# Formulation 1: Baseline

In [ ]:
exps_base = []
for n_memory_samples in mem_sizes:
    n_enc = f"cifar100_m{n_memory_samples}_cutr34_{BACKBONE_BLOCK}_splitr34_{BACKBONE_BLOCK}__s{SEED}"
    l_enc =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--encoder", "cutr34",
            "--encoding_block", str(BACKBONE_BLOCK),
            "--compressor", "none",
            "--backbone", "resnet34",
            "--backbone_block", str(BACKBONE_BLOCK),
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
            "--exp_name", n_enc]
    exps_base.append(" ".join(l_enc))

for n_memory_samples in mem_sizes:
    n_unenc = f"cifar100_m{n_memory_samples}_resnet34__s{SEED}"
    l_unenc =  ["python3 src/main.py",
                "--dataset", "CIFAR100",
                "--num_classes_per_task", "5",
                "--num_tasks", "20",
                "--seed", str(SEED),
                "--memory_size", str(n_memory_samples),
                "--num_passes", "128",
                "--sampler", "greedy_sampler",
                "--encoder", "none",
                "--compressor", "none",
                "--backbone", "resnet34",
                "--backbone_block", "0",
                "--data_dir", DATA_DIR,
                "--log_dir", LOG_DIR,
                "--exp_name", n_unenc]
    exps_base.append(" ".join(l_unenc))

write_save('cifar100_rescon_base.sh', '\n'.join(exps_base))

# Formulation 2: Thinning


In [ ]:
compression_factors = [0.5, 0.8, 0.9, 0.95]

In [ ]:
exps_thinning = []
for n_memory_samples in mem_sizes:
    for this_compression_factor in compression_factors:
        
        n = f"cifar100_m{n_memory_samples}_cutr34_{BACKBONE_BLOCK}_thinning{int(this_compression_factor*100)}_splitr34_{BACKBONE_BLOCK}__s{SEED}"
        l =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--encoder", "cutr34",
            "--encoding_block", str(BACKBONE_BLOCK),
            "--compressor", "thinning",
            "--compression_factor", str(this_compression_factor),
            "--backbone", "resnet34",
            "--backbone_block", str(BACKBONE_BLOCK),
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
            "--exp_name", n]
        exps_thinning.append(" ".join(l))

for n_memory_samples in mem_sizes:
    for this_compression_factor in compression_factors:
        n = f"cifar100_m{n_memory_samples}_thinning{int(this_compression_factor*100)}_resnet34__s{SEED}"
        l =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--encoder", "none",
            "--compressor", "thinning",
            "--compression_factor", str(this_compression_factor),
            "--backbone", "resnet34",
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
            "--exp_name", n]
        exps_thinning.append(" ".join(l))

write_save('cifar100_rescon_thinning.sh', '\n'.join(exps_thinning))

# Formulation 3: Quantization

We have 4 loops for all combinations:

* with and without the encoder
* with local and transfer compression

In [ ]:
n_states_list = [2, 4, 8, 16, 32]

In [ ]:
exps_quantization = []

# encoding and local quantization
for n_memory_samples in mem_sizes:
    for n_states in n_states_list:
        n = f"cifar100_m{n_memory_samples}_cutr34_{BACKBONE_BLOCK}_quantization{n_states}_splitr34_{BACKBONE_BLOCK}__s{SEED}"
        l =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--encoder", "cutr34",
            "--encoding_block", str(BACKBONE_BLOCK),
            "--compressor", "quantization",
            "--n_states", str(n_states),
            "--backbone", "resnet34",
            "--backbone_block", str(BACKBONE_BLOCK),
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
            "--exp_name", n]
        exps_quantization.append(" ".join(l))

# no encoding and local quantization
for n_memory_samples in mem_sizes:
    for n_states in n_states_list:    
        n = f"cifar100_m{n_memory_samples}_quantization{n_states}_resnet34__s{SEED}"
        l =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--encoder", "none",
            "--compressor", "quantization",
            "--n_states", str(n_states),
            "--backbone", "resnet34",
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
            "--exp_name", n]
        exps_quantization.append(" ".join(l))

# encoding and global quantization
for n_memory_samples in mem_sizes:
    for n_states in n_states_list:
        n = f"cifar100_m{n_memory_samples}_cutr34_{BACKBONE_BLOCK}_quantization{n_states}_splitr34_{BACKBONE_BLOCK}__s{SEED}_transfer"
        l =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--encoder", "cutr34",
            "--encoding_block", str(BACKBONE_BLOCK),
            "--compressor", "quantization",
            "--n_states", str(n_states),
            "--strategy", "tiny_imagenet_transfer",
            "--backbone", "resnet34",
            "--backbone_block", str(BACKBONE_BLOCK),
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
            "--exp_name", n]
        exps_quantization.append(" ".join(l))

# no encoding and global quantization
for n_memory_samples in mem_sizes:
    for n_states in n_states_list:    
        n = f"cifar100_m{n_memory_samples}_quantization{n_states}_resnet34__s{SEED}_transfer"
        l =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--encoder", "none",
            "--compressor", "quantization",
            "--n_states", str(n_states),
            "--strategy", "tiny_imagenet_transfer",
            "--backbone", "resnet34",
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
            "--exp_name", n]
        exps_quantization.append(" ".join(l))

write_save('cifar100_rescon_quantization.sh', '\n'.join(exps_quantization))

# Formulation 4: conv. Autoencoder

We only get good performance for 2 blocks. This might be because the the size of the latent vector is to small for lager network-sizes. 2 is used as a default for all the other experiments.

Also keep in mind that the conv. ae. does not work with encoded inputs

In [ ]:
AE_N_BLOCKS = 2
latent_clannels_list = [1, 2, 4, 8, 16]

In [ ]:
exps_autoencoder = []

# without encoder
for n_memory_samples in mem_sizes:
    for this_latent_channels in latent_clannels_list:
        n = f"cifar100_m{n_memory_samples}_convae{this_latent_channels}_resnet34__s{SEED}"
        l =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--encoder", "none",
            "--compressor", "convae",
            "--n_blocks", str(AE_N_BLOCKS),
            "--latent_channels", str(this_latent_channels),
            "--backbone", "resnet34",
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
            "--exp_name", n]
        exps_autoencoder.append(" ".join(l))

write_save('cifar100_rescon_convae.sh', '\n'.join(exps_autoencoder))

# Formulation 5: FC Autoencoder

In [ ]:
bottleneck_sizes = [2, 4, 8, 16, 32, 64]

In [ ]:
exps_fcae_enc = []
exps_fcae_none = []

for n_memory_samples in mem_sizes:
    for bottleneck_size in bottleneck_sizes:
        l =  ["python3 src/main.py",
            "--dataset", "CIFAR100",
            "--num_classes_per_task", "5",
            "--num_tasks", "20",
            "--seed", str(SEED),
            "--memory_size", str(n_memory_samples),
            "--num_passes", "128",
            "--sampler", "greedy_sampler",
            "--compressor", "fcae",
            "--bottleneck_neurons", str(bottleneck_size),
            "--data_dir", DATA_DIR,
            "--log_dir", LOG_DIR,
        ]

        n_no_enc = f"cifar100_m{n_memory_samples}_fcae{bottleneck_size}_resnet34__s{SEED}"
        n_enc = f"cifar100_m{n_memory_samples}_cutr34_{BACKBONE_BLOCK}_fcae{bottleneck_size}_resnet34__s{SEED}"

        l_no_enc = l + [
            "--encoder", "none",
            "--backbone", "resnet34",
            "--exp_name", n_no_enc
        ]
        l_enc = l + [
            "--encoder", "cutr34",
            "--encoding_block", str(BACKBONE_BLOCK),
            "--backbone", "resnet34",
            "--backbone_block", str(BACKBONE_BLOCK),
            "--exp_name", n_enc        
        ]

        exps_fcae_none.append(' '.join(l_no_enc))
        exps_fcae_enc.append(' '.join(l_enc))
exps_fcae = exps_fcae_none + exps_fcae_enc

write_save('cifar100_rescon_fcae.sh', '\n'.join(exps_fcae))

# Write one file for everything

In [ ]:
all_exps = exps_base + exps_thinning + exps_quantization + exps_autoencoder + exps_fcae
write_save('cifar100_rescon_all.sh', '\n'.join(all_exps))

# Results

CIFAR10-Dataset

| cut...                    | output shape | output size | bits (bytes) to adress output coordinates |
|---------------------------|--------------|-------------|-------------------------------------------|
| after Block 1             | 8x8x64       |  4096       | 12 (2)                                    |
| after Block 2             | 4x4x128      |  2048       | 11 (2)                                    |
| after Block 3             | 2x2x256      |  1024       | 10 (2)                                    |
| after Block 4             | 1x1x512      |   512       |  9 (2)                                    |

We assume

* one float takes up 4 Byte of Memory
* uints are used for output-coordinates

In [ ]:
from pathlib import Path

parent_dir = Path(LOG_DIR)
MODEL_SIZE_MB = 0
OUTPUT_SIZE_ENCODED = 1024
OUTPUT_SIZE_UNENCODED = 32*32*3
FLOAT_SIZE_BYTE = 4
UINT_SIZE_BYTE = 1
COORDINATE_SIZE_BYTE = 2

In [ ]:
import re

log_paths = [Path(d, 'checkpoint.log') for d in parent_dir.glob('cifar100_*')]
c = []

for exp in log_paths:
    with open(exp) as infile:
        loglines = infile.read().splitlines()

    try:
        mem_size = int(re.findall(r"memory_size=(\d+)", loglines[0])[0])
        final_acc = float(re.findall(r"Acc: \[(.*?)\]", loglines[-1])[0])
        compressor = re.findall(r"compressor=\'(.*?)\'", loglines[0])[0]
        encoder = re.findall(r"encoder=\'(.*?)\'", loglines[0])[0]
    except IndexError:
        print(f'could not parse {exp}')
        continue

    compressor_param = None

    if compressor == 'thinning':
        compressor_param = float(re.findall(r"compression_factor=(.*?),", loglines[0])[0])
        compressor_name = f'{compressor} ({compressor_param})'
    elif compressor == 'quantization':
        compressor_param = int(re.findall(r"n_states=(\d+)", loglines[0])[0])
        try:
            strategy = re.findall(r"strategy=\'(.*?)\'", loglines[0])[0]
            assert strategy == 'tiny_imagenet_transfer' or strategy == 'local'
            if strategy == 'tiny_imagenet_transfer':
                compressor_new = 'quantization transfer'
            elif strategy == 'local':
                compressor_new = 'quantization local'
            else:
                raise ValueError('Unknown Quantization Stragegy: ' + strategy)
        except IndexError:
            compressor_new = 'quantization local'
        compressor_name = f'{compressor} ({compressor_param})'
        compressor = compressor_new
    elif compressor == 'fcae':
        compressor_param = int(re.findall(r"bottleneck_neurons=(\d+)", loglines[0])[0])
        compressor_name = f'Autoencoder (FC, {compressor_param})'
    elif compressor == 'convae':
        compressor_param = int(re.findall(r"latent_channels=(\d+)", loglines[0])[0])
        compressor_name = f'Autoencoder (Conv., {compressor_param})'
    elif compressor == 'none':
        compressor_name = 'no compression'
        compressor_param = 0
    else:
        raise ValueError(f'Unknown Compressor {compressor}')

    if encoder == 'cutr34':
        encoding_block = int(re.findall(r"encoding_block=(\d+)", loglines[0])[0])
        output_size = OUTPUT_SIZE_ENCODED
        encoder_name = f'CutR34({encoding_block})'
    elif encoder == 'none':
        output_size = OUTPUT_SIZE_UNENCODED
        encoder_name = 'no encoding'
    else:
        raise ValueError('Unknown Encoder')

    c.append({
        'mem_size': mem_size,
        'final_acc': final_acc,
        'output_size': output_size,
        'encoder': encoder,
        'encoder_name': encoder_name,
        'compressor': compressor,
        'compressor_name': compressor_name,
        'compressor_param': compressor_param,
    })

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records(c)
df['mem_size_mb'] = 0
df['byte_for_datatype'] = 0
df.loc[df['encoder']=='none', 'bytes_for_datatype'] = UINT_SIZE_BYTE
df.loc[df['encoder']=='cutr34', 'bytes_for_datatype'] = FLOAT_SIZE_BYTE

In [ ]:
df['sort_idx'] = 0
df.loc[df.compressor=='none', 'sort_idx'] = 0
df.loc[df.compressor=='thinning', 'sort_idx'] = 1
df.loc[df.compressor=='quantization local', 'sort_idx'] = 2
df.loc[df.compressor=='quantization transfer', 'sort_idx'] = 3
df.loc[df.compressor=='fcae', 'sort_idx'] = 4
df.loc[df.compressor=='convae', 'sort_idx'] = 5

Baseline

In [ ]:
df.loc[df['compressor']=='none', 'mem_size_mb'] = \
    df.loc[df['compressor']=='none', 'bytes_for_datatype'] \
    * df.loc[df['compressor']=='none', 'output_size'] \
    * df.loc[df['compressor']=='none', 'mem_size'] \
    / (1024*1024) \
    + MODEL_SIZE_MB

Thinning

In [ ]:
df['n_elements_per_sample'] = 0
df['sample_size_byte'] = 0

df.loc[df.compressor=='thinning', 'n_elements_per_sample'] = \
    df.loc[df.compressor=='thinning', 'output_size'] \
    * (1 - df.loc[df.compressor=='thinning', 'compressor_param'])
df.loc[df.compressor=='thinning', 'sample_size_byte'] = \
    df.loc[df.compressor=='thinning', 'n_elements_per_sample'] \
    * df.loc[df.compressor=='thinning', 'bytes_for_datatype'] \
    + df.loc[df.compressor=='thinning', 'n_elements_per_sample'] \
    * COORDINATE_SIZE_BYTE
df.loc[df.compressor=='thinning', 'mem_size_mb'] = \
    df.loc[df.compressor=='thinning', 'sample_size_byte'] \
    * df.loc[df.compressor=='thinning', 'mem_size'] \
    / (1024*1024) \
    + MODEL_SIZE_MB
df.drop(columns=['n_elements_per_sample', 'sample_size_byte'], inplace=True)


Quantization

In [ ]:
from ctypes.wintypes import FLOAT
import numpy as np
df['compressed_data'] = 0
df['inverval_centers'] = 0

mask = (df.compressor=='quantization local') | (df.compressor=='quantization transfer')
df.loc[mask, 'compressed_data'] = \
    df.loc[mask, 'mem_size'] \
    * np.ceil(
        np.ceil(np.log2(df.loc[mask, 'compressor_param']))
        * df.loc[mask, 'output_size']
        / 8
    )

df.loc[df['compressor']=='quantization local', 'inverval_centers'] = \
    df.loc[df['compressor']=='quantization local', 'mem_size'] \
    * df.loc[df['compressor']=='quantization local', 'compressor_param'] \
    * FLOAT_SIZE_BYTE

df.loc[df['compressor']=='quantization transfer', 'inverval_centers'] = \
    df.loc[df['compressor']=='quantization transfer', 'compressor_param'] \
    * FLOAT_SIZE_BYTE

df.loc[mask, 'mem_size_mb'] = \
    MODEL_SIZE_MB \
    + (
        df.loc[mask, 'inverval_centers']
        + df.loc[mask, 'compressed_data']
    ) / (8*1024*1024)

df.drop(columns=['compressed_data', 'inverval_centers'], inplace=True)

Convolutional Autoencoder. This was only done using unencoded inputs

In [ ]:
ae_size_mb = {
    1: 0.00452423095703125,
    2: 0.0056304931640625,
    4: 0.007843017578125,
    8: 0.01226806640625,
    16: 0.0211181640625,
}

df['convae_size'] = df['compressor_param'].astype('int')
df.loc[df['compressor']=='convae', 'convae_size'] = df.loc[df['compressor']=='convae', 'convae_size'].replace(ae_size_mb)
df.loc[df['compressor']=='convae', 'mem_size_mb'] = \
    MODEL_SIZE_MB \
    + df.loc[df['compressor']=='convae', 'convae_size'] \
    + ( 
        df.loc[df['compressor']=='convae', 'mem_size']
        * 8 * 8 * df.loc[df['compressor']=='convae', 'compressor_param']
        * FLOAT_SIZE_BYTE
        / (1024*1024)
    )

df.drop(columns=['convae_size'], inplace=True)

Fully Connected Autoencoder

In [ ]:
SIZE_FCAE_NONE_MB = {
    64: 52.08887481689453,
    32: 43.0804443359375,
    16: 36.295753479003906,
    8: 30.972145080566406,
    4: 26.713584899902344,
    2: 23.205177307128906,
}
SIZE_FCAE_CUTR_MB = {
    64: 8.331954956054688,
    32: 6.5640106201171875,
    16: 5.3397216796875,
    8: 4.450263977050781,
    4: 3.7687225341796875,
    2: 3.2361679077148438,
}

df['fcae_size'] = df['compressor_param'].astype('int')
df.loc[(df['compressor']=='fcae') & (df['encoder']=='none'), 'fcae_size'] = df.loc[(df['compressor']=='fcae') & (df['encoder']=='none'), 'fcae_size'].replace(SIZE_FCAE_NONE_MB)
df.loc[(df['compressor']=='fcae') & (df['encoder']=='cutr34'), 'fcae_size'] = df.loc[(df['compressor']=='fcae') & (df['encoder']=='cutr34'), 'fcae_size'].replace(SIZE_FCAE_CUTR_MB)
df.loc[df['compressor']=='fcae', 'mem_size_mb'] = \
    df.loc[df['compressor']=='fcae', 'fcae_size'] \
    + (
        df.loc[df['compressor']=='fcae', 'mem_size']
        * df.loc[df['compressor']=='fcae', 'compressor_param']
        * FLOAT_SIZE_BYTE
        / (1024*1024)
    )
df.drop(columns=['fcae_size'], inplace=True)

Plot

In [ ]:
import plotly.express as px
from plot_utils import science_template

config = {
    'displaylogo': False,
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'rescon',
        'scale': 3 # Multiply title/legend/axis/canvas sizes by this factor
    }
}

fig = px.line(
    df.sort_values(['sort_idx', 'mem_size_mb']),
    x='mem_size_mb',
    y='final_acc',
    color='compressor_name',
    facet_row='encoder_name',
    facet_col='compressor',
    markers=True,
    log_x=True,
    template=science_template,
    title='CIFAR100',
    hover_data={
        'mem_size': True
    },
    labels={
        'mem_size': 'Number of Memory Slots',
        'final_acc': 'Accuracy',
        'mem_size_mb': 'Total Storage Comsumption [MiB]',
        'encoder_name': 'Encoder',
        'compressor_name': 'Compression Strategy',
        'compressor': 'Compressor',
        'fcae': 'Fully Connected Autoencoder'
    })
fig.show(renderer='browser', config=config)


Use Matplotlib so we have more options

In [ ]:
import matplotlib.pyplot  as plt

df.sort_values(['sort_idx', 'mem_size_mb'], inplace=True)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(30, 7))
fig.subplots_adjust(hspace=0.4, wspace=0.2)

c_names = {
    'thinning': 'Thinning',
    'quantization local': 'Quantization (local)',
    'quantization transfer': 'Quantization (transfer)',
    'fcae': 'Fully Connected Autoencoder',
    'convae': 'Convolutional Autoencoder',
    'none': 'No Compression'
}

colors = [
        '#636efa',
        '#EF553B',
        '#00cc96',
        '#ab63fa',
        '#FFA15A',
        '#19d3f3',
        '#FF6692',
        '#B6E880',
        '#FF97FF',
        '#FECB52'
]

for i_comp, comp in enumerate(df.compressor.unique()):
    mem_min = df.loc[df['compressor']==comp, 'mem_size_mb'].min()
    mem_max = df.loc[df['compressor']==comp, 'mem_size_mb'].max()
    for i_enc, enc in enumerate(df.encoder.unique()):
        if comp == 'convae' and enc == 'cutr34':
            axes[i_enc, i_comp].remove()
            continue
        compressor_params = df.loc[(df['encoder']==enc) & (df['compressor']==comp), 'compressor_param'].unique()
        ymax = df.loc[df['encoder']==enc, 'final_acc'].max() * 1.1
        for icp, cp in enumerate(compressor_params):
            df.loc[(df['encoder']==enc) & (df['compressor']==comp) & (df['compressor_param']==cp)].plot(
                x='mem_size_mb',
                y='final_acc',
                ax=axes[i_enc, i_comp],
                logx=True,
                xlim=(mem_min*0.8, mem_max*1.2),
                ylim=(-0.03, ymax),
                grid=True,
                marker='o',
                color=colors[icp]
            )

        if enc == 'none':
            if comp == 'none':
                title = 'GDumb'
            else:
                title = 'ResNet-34 + ' + c_names[comp]
        else:
            enc_name = df.loc[(df['encoder']==enc) & (df['compressor']==comp), 'encoder_name'].unique()[0]
            title = f"{enc_name} + {c_names[comp]}"
        axes[i_enc, i_comp].set_title(title)
        if comp != 'thinning':
            compressor_params = [int(x) for x in compressor_params]
        axes[i_enc, i_comp].axhline(y=0.01, xmin=0, xmax=1, c="k", linewidth=1, zorder=0, linestyle='--')
        if (enc, comp) == ('none', 'fcae'):
            axes[i_enc, i_comp].legend(compressor_params, loc='upper left')
        elif (enc, comp) == ('cutr34', 'fcae'):
            axes[i_enc, i_comp].legend(compressor_params, loc='upper right')
        else:
            axes[i_enc, i_comp].legend(compressor_params)
        axes[i_enc, i_comp].set_xlabel("")
        axes[i_enc, i_comp].set_ylabel("")
        if i_comp==0:
            axes[i_enc, i_comp].legend().remove()
            axes[i_enc, i_comp].set_ylabel("Accuracy")
        if i_enc==1:
            axes[i_enc, i_comp].set_xlabel("Memory Size [MiB]")




In [ ]:
fig.savefig('plot.pdf')

# Best Performance for Memory

In [ ]:
view = df.sort_values('mem_size_mb')
view['method'] = df['compressor'].replace(c_names) + ' k=' + view['compressor_param'].astype(int).astype(str)
view.loc[df['compressor'] == 'none', 'method'] = df.loc[df['compressor'] == 'none', 'compressor'].replace(c_names)
view.loc[df['compressor'] == 'thinning', 'method'] = df.loc[df['compressor'] == 'thinning', 'compressor'].replace(c_names) + ' k=' + view.loc[df['compressor'] == 'thinning', 'compressor_param'].astype(str)

In [ ]:
acc = 0
mem_sizes = view['mem_size_mb'].unique()

for this_mem_size in mem_sizes:
    cur_max = view.loc[view['mem_size_mb'] == this_mem_size, 'final_acc'].max()
    if cur_max > acc:
        acc = cur_max
    view = view.drop(
        view[
            (view['mem_size_mb'] == this_mem_size) & (view['final_acc'] < acc)
        ].index
    )

In [ ]:
import plotly.graph_objects as go

fig = px.scatter(
    view.loc[(view['compressor'] == 'quantization local') | (view['compressor'] == 'quantization transfer')],
    x='mem_size_mb',
    y='final_acc',
    color=np.log10(view.loc[(view['compressor'] == 'quantization local') | (view['compressor'] == 'quantization transfer'), 'compressor_param']),
    symbol='compressor',
    log_x=True,
    hover_data={
        'mem_size': True,
        'encoder': True
    },
    template=science_template,
    labels={
        'mem_size': 'Number of Memory Slots',
        'final_acc': 'Accuracy',
        'mem_size_mb': 'Total Storage Comsumption [MiB]',
        'encoder_name': 'Encoder',
        'compressor_name': 'Compression Strategy',
        'compressor': 'Compressor',
        'fcae': 'Fully Connected Autoencoder',
    },
    color_continuous_scale='Agsunset'
)

fig.add_trace(
    go.Scatter(
        x=view.loc[(view['compressor'] == 'thinning'), 'mem_size_mb'],
        y=view.loc[(view['compressor'] == 'thinning'), 'final_acc'],
        mode='markers',
        marker_color='black',
        marker_symbol='x',
        name='Thinning (k=0.5)'
    )
)
fig.add_trace(
    go.Scatter(
        x=view.loc[(view['compressor'] == 'none'), 'mem_size_mb'],
        y=view.loc[(view['compressor'] == 'none'), 'final_acc'],
        mode='markers',
        marker_color='black',
        marker_symbol='cross',
        name='GDumb'
    )
)

fig.update_layout({'legend_title_text': ''})
fig.update(layout_coloraxis_showscale=False)
fig.update_traces(
    marker={
        'size': 8,
    },
)
fig.update_yaxes(range=[0, 0.5])
config = {
    'displaylogo': False,
    'toImageButtonOptions': {
        'format': 'svg', # one of png, svg, jpeg, webp
    }
}

fig.show(config=config, renderer='browser')

In [ ]:
view.loc[(view['compressor'] != 'quantization local') & (view['compressor'] != 'quantization transfer')]